# Convert .dcm to .png
1. image_level : 512 size, study_level : 600 size  
    efnb7 학습 및 yolov5 모델 학습을 위한 것.
2. image_level : 1280 size  
    yolov5x6 model 학습을 위한 데이터셋

[[Notebook] Reference](https://www.kaggle.com/h053473666/512-img-png-600-study-png)

In [18]:
!conda install gdcm -c conda-forge -y
# gdcm : for reading .dcm files

Solving environment: done

# All requested packages already installed.



In [27]:
import os

from glob import glob
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import shutil

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [20]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == 'MONOCHROME1':
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    if keep_ratio:
        im.thumnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

# For train set image

In [21]:
# study_level
train = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
display(train.head(), train.shape)

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


(6054, 5)

In [22]:
# image_level
train = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
display(train.head(), train.shape)

,id,boxes,label,StudyInstanceUID
0,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75
1,000c3a3f293f_image,NaN,none 1 0 0 1 1,ff0879eb20ed
2,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7
3,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2
4,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e


(6334, 4)

구조상, train_study_level.csv 에서 id 컬럼의 앞 부분이 train folder 의 각 folder 를 가짐.  
그 폴더 내부에 train_image_level.csv 의 StudyInstanceUID 컬럼에 해당하는 이름의 폴더들이 있음.  
그 각 폴더 내부에 id 컬럼의 이름에 해당하는 image 들이 존재

In [23]:
train['id'].nunique(), train['StudyInstanceUID'].nunique()

(6334, 6054)

In [24]:
# sample read_file
path = '../input/siim-covid19-detection/train/ae3e63d94c13/288554eb6182/e00f9fe0cce5.dcm'
dicom = pydicom.read_file(path)
dicom

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 198
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Computed Radiography Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.3680043.10.474.5487382592917001697609898110013378421
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.2.40.0.13.1.1.1
(0002, 0013) Implementation Version Name         SH: 'dcm4che-1.4.38'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY']
(0008, 0016) SOP Class UID                       UI: 03a65300fa41
(0008, 0018) SOP Instance UID                    UI: e00f9fe0cce5
(0008, 0020) S

## os.walk
```py
for path, directory, files in os.walk(path):
```
path, directory, files 리턴  
만약 path 내에 폴더로만 구성되어 있다면, 첫 번째 리턴은 path = path, directory = folders, files = None  
두 번째 부터는 folders 를 하나씩 path 로 삼아 탐색  


In [28]:
split = 'train'

# folder
save_dir = f"/kaggle/working/tmp/{split}/" 

if os.path.isdir(save_dir):
    shutil.rmtree(save_dir)
    os.makedirs(save_dir, exist_ok = True)

In [29]:
# 1. convert image_level
image_level = True

if image_level:
    # folder for study_level
    save_dir = f"/kaggle/working/tmp/{split}/image/"
    os.makedirs(save_dir, exist_ok = True)
    
    image_size = 1280
    for dirname, _, filenames in tqdm(os.walk(f"/kaggle/input/siim-covid19-detection/{split}")):
        for file in filenames:
            # set 'keep_ratio=True' to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size = image_size)
            im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
            
# 다 뒤져서 image 저장
# 12386 image 1280 size : 40 min (mistake -> image_size = 1)
# 12386 image 1280 size : 107 min

0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pydicom/pixel_data_handlers/numpy_handler.py:341: UserWarning: The length of the pixel data in the dataset (13262360 bytes) indicates it contains excess padding. 216296 bytes will be removed from the end of the data
  warnings.warn(msg)


In [33]:
# zip
#!zip -r image_1280.zip ./tmp/train/image
glob('./*')

['./__notebook_source__.ipynb', './tmp', './image_1280.zip']

In [ ]:
# 2. convert study_level
study_level = False
if study_level:
    # folder for study_level
    save_dir = f"/kaggle/working/tmp/{split}/study/'
    os.makedirs(save_dir, exist_ok = True)
    
    count = 0
    for dirname, _, filenames in tqdm(os.walk(f"/kaggle/input/siim-covid19-detection/{split}")):
        for file in filenames:
            # set 'keep_ratio=True' to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            #im = resize(xray, size = 600)
            study = dirname.split('/')[-2] + '_study.png'
            #im.save(os.path.join(save_dir, study))
            if len(filenames) > 1:
                print(f"filenames : {filenames}")
            else:
                count += 1
            if count % 100 == 0:
                print(count)
            
# dirname : /kaggle/input/siim-covid19-detection/train/StudyInstanceUID/id(in train_study_level.csv)
# StudyInstanceUID 가 큰 폴더(내부폴더X)
# 이와 같은 방식으로 하면,
# train 의 폴더는 총 6054개(study_level), 이미지 파일은 총 6334개(image_level) 인데,
# 그러면 어떤 연구 폴더에는 이미지가 2개 이상이 들어가 있다.
# image_level 을 구성할때는, im.save 를 할 때, 그 이미지 이름에 _image.png 만 바꿔서 저장하므로 겹칠일이 없다.
# study_level 를 구성할때는, 각 파일 이름이 '폴더이름 + _study.png' 가 되므로, 각 폴더의 마지막 이미지가 저장된다.
# 즉, study_level 에서는 잘못된 이미지, 오류 이미지들이 있는데도 그것들을 고려하지 않고 마지막 이미지 기준으로 저장된다.

# 추후에 이것을 뜯어서 study 단에서 이미지 갯수가 2개 이상일 경우, 하나하나 확인해봐야겠다. 
# study inference 에 문제가 되므로.

# 그리고 다시 문제를 정의하자면, study level 은 각 폴더가 study_level 이다.
# 연구를 위해 같다고 생각되는 이미지를 모은 것이고, 그 study_level 이 어떤 증상을 나타내고 있는지 분류하는 것.
# (study 단에서 겹치는 분류 카테고리가 동시에 존재하지는 않는다. 즉, study_level 기준 하에서는 같은 증상이라는 것)
# image_level 은 그냥 모든 이미지에 대해 bounding box 를 추출하는 것.

# negative 와 none 의 관계도 잘 알아봐야겠다.
# 모델의 물체 인식도 또한 문제가 되고, 모델과 별개로 의학적 분석에 있어서도 어떤지 봐야겠다.
# 이는, boxes 가 발견되었다고 해서 모델이 음성이 아니라고 판단할 수 없다.(모델의 한계)
#*그렇다면 의학적으로 box가 발견되었다고 해서 꼭 음성이 아니라는 보장(꼭 양성이라고 단정지을 수 없다는 것)이 없다고 해석해야 하나?. 
# 양성이라고 해서(음성이 아니라고 해서) 모델이 꼭 box 를 발견하지는 않는다.
#*단, 엄밀히 양성일 경우 반드시 box 가 존재해야 하지 않을까?
# 여기에는 annotation 문제도 같이 고려해야 한다.
# 이렇게 negative 와 none 의 관계는 엄밀히 고려해야 할 부분이 5가지가 된다.


### image 변환하여 저장하는 Notebook   
[[Notebook] 이미지 변환 저장 참고 노트북](https://www.kaggle.com/madquer/convert-512-img-600-study-png/edit)  
이 노트북의 변환과정에 따르면,  
image_level 에서는 위의 4개 모두가 저장될 것이고,    
study_level 에서는 위 사진 중 마지막 사진이 study_level 용 데이터로 저장될 것이다.   
정말 엄밀히 하자면, image_level 은 어차피 모든 boxes 를 다 예측하는데, study_level 은 이 중 하나만 선택된다.   
**image_level 에 따라 맞춰가면서 학습에 이용할 study_level 을 선정하는 것도 중요하겠다.**  
**또한, 코드를 수정하여 4가지 사진 모두에 대해 학습시켜 볼 수 도 있겠다.**

In [ ]:
# study_level 폴더 안에도 여러 폴더가 존재하는데, 위에서 따로 추출된 것들은 내부 폴더 중 한 폴더 안에 여러개가 존재할 때만 출력한 것.
# 즉 구조는, study_level folder 6054 개 그 각 폴더 내에 여러 폴더들이 있고, 그 폴더들 내에 이미지들이 있다.
# 마지막 단의 폴더 안에 이미지가 여러개 있는 것도 있다. 

# 위의 study_level 은, study_level 최상단 폴더 이름을 기준으로, 그 내부에 있는 이미지는 몽땅그려 하나로 출력한 것.
# study_level 하나 내부에 있는 여러 이미지 중 마지막 이미지가 study_level 용 이미지로 결정된다.
# 그래서, study_level 이미지 중 여러개 있는 것들을 서로 비교하여 학습에 다른 방식으로 활용해야겠다.
# 왜냐면, study_level 단 이미지 중 어떤 것들은 품질이 좋지않고,
# 실제로 그 이미지들은 그러한 맥락에서 여러번 찍힌 것이기 때문이다.

# For test set image

In [ ]:
print('start')

In [ ]:
split = 'test'

# folder
save_dir = f"/kaggle/working/tmp/{split}/" 
os.makedirs(save_dir, exist_ok = True)

# folder for image_level
save_dir = f"/kaggle/working/tmp/{split}/image/"
os.makedirs(save_dir, exist_ok = True)

# 1. convert image_level
for dirname, _, filenames in tqdm(os.walk(f"/kaggle/input/siim-covid19-detection/{split}")):
    for file in filenames:
        # set 'keep_ratio=True' to have original aspect ratio : current -> False
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size = 512)
        im.save(os.path.join(save_dir, file.replace('.dcm', '_image.png')))
# 다 뒤져서 image 저장

In [ ]:
# folder for study_level
save_dir = f"/kaggle/working/tmp/{split}/study/"
os.makedirs(save_dir, exist_ok = True)

# 2. convert study_level
count = 0
for dirname, _, filenames in tqdm(os.walk(f"/kaggle/input/siim-covid19-detection/{split}")):
    for file in filenames:
        # set 'keep_ratio=True' to have original aspect ratio : current -> False
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size = 600)
        study = dirname.split('/')[-2] + '_study.png'
        im.save(os.path.join(save_dir, study))

In [ ]:
# zip & download
!zip -r /kaggle/working/test.zip /kaggle/working/tmp/test